In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from scipy import stats,sparse
from sklearn.base import TransformerMixin
from datetime import datetime as dt
from math import isnan
from numpy import ma
import pickle as pickle
import xgboost as xgb
import time
from pandas import *
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier 

import json
from sklearn.metrics import roc_curve, auc
from re import sub
from collections import defaultdict
import scipy as sp

from sklearn.cross_validation import StratifiedKFold,cross_val_score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import linear_model
from sklearn import metrics

from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from scipy import sparse
from sklearn.preprocessing import OneHotEncoder,StandardScaler

In [25]:
FeaturesSelected=np.load('./mostrecentinput/SDGLogistic_Oct172015032138_AUC_0.768923541152.p')
X=np.load('./mostrecentinput/nxtrain_standard_original0.npy')
X1= np.load('./mostrecentinput/nxtrain_standard_derived0.npy')
X2=pd.read_pickle("./mostrecentinput/time_series_derived_standard_train2.dat")
X3=pd.read_pickle("./mostrecentinput/time_series_original_standard_train2.dat")
X4=pd.read_pickle("./mostrecentinput/cat_numeric_th60_standard_train2.dat")
X5=pd.read_pickle("./mostrecentinput/cat_le_train2.dat") 
xtrain_cat=np.hstack((X,X1,X2,X3,X4,X5))
del X, X1,X2,X3,X4,X5
y=pickle.load(open("./mostrecentinput/ytrain2.dat","rb"))
xtrain_quant=xtrain_cat[:,FeaturesSelected]
xtrain_cat  =np.delete(xtrain_cat,FeaturesSelected,axis=1)

In [26]:
X_test=np.load('./mostrecentinput/nxtest_standard_original0.npy')
X_test1= np.load('./mostrecentinput/nxtest_standard_derived0.npy')
X_test2=pd.read_pickle("./mostrecentinput/time_series_derived_standard_test2.dat")
X_test3=pd.read_pickle("./mostrecentinput/time_series_original_standard_test2.dat")
X_test4=pd.read_pickle("./mostrecentinput/cat_numeric_th60_standard_test2.dat")
X_test5=pd.read_pickle("./mostrecentinput/cat_le_test2.dat") 
xtest_cat=np.hstack((X_test,X_test1,X_test2,X_test3,X_test4,X_test5))

xtest_quant=xtest_cat[:,FeaturesSelected]
del X_test, X_test1,X_test2,X_test3,X_test4,X_test5
xtest_cat  =np.delete(xtest_cat,FeaturesSelected,axis=1)
TestID=pd.read_pickle("./mostrecentinput/TestID.p")

# First round xgBoost

In [30]:
KFoldNumber=3
num_boost=100

ColumnKFoldNumber=50
XX=np.zeros(xtrain_cat0.shape[1]).astype(int)
KfoldColumn= StratifiedKFold(XX, ColumnKFoldNumber,shuffle=True, random_state=42)

kfoldColumnnum=-1

ypreds_train_Full_wl=pd.DataFrame()
ypreds_test_Full_wl=pd.DataFrame()


for trainCol,cvCol in KfoldColumn:
    kfoldColumnnum=kfoldColumnnum+1
    xtrain_cat=xtrain_cat0[:,cvCol]
    xtest_cat=xtest_cat0[:,cvCol]  


    kfold = StratifiedKFold(y, KFoldNumber,shuffle=True, random_state=42)
    ypreds_test_wl=pd.DataFrame()
    ypreds_train_wl=pd.DataFrame()
    ypreds_cvindex_wl=pd.DataFrame()
    AUCValues=np.zeros(KFoldNumber)

    kfoldnum=0

    train_predicted=np.zeros(y.shape)


    params= {'max_depth':14, 
                 'eta':0.01, 
                 'objective':'binary:logistic', 
                 'subsample':0.6,
                 'colsample_bytree':0.6,
                 'eval_metric': 'auc'}

    for train,cv in kfold:
        kfoldnum=kfoldnum+1
        cv_index=np.zeros(y.shape)

        dtrain = xgb.DMatrix(xtrain_cat[train,:], label=y[train])
        clf = xgb.train(params=params,dtrain=dtrain, num_boost_round=num_boost)    
        del dtrain
        dfulltrain = xgb.DMatrix(xtrain_cat, label=y)
        preds_fulltrain = clf.predict(dfulltrain)
        del dfulltrain
        dfulltest = xgb.DMatrix(xtest_cat, label=np.zeros(xtest_cat.shape[0]))
        preds_fulltest = clf.predict(dfulltest)
        del dfulltest
        preds_cv=preds_fulltrain[cv]
        
        cv_index[cv]=1
        AUC1=metrics.roc_auc_score(y[cv], preds_cv)
        ypreds_test_wl['CV'+ str(kfoldnum)] = preds_fulltest
        ypreds_train_wl['CV'+ str(kfoldnum)] = preds_fulltrain
        ypreds_cvindex_wl['CV'+ str(kfoldnum)] = cv_index
        AUCValues[kfoldnum-1]=float(AUC1)
        print(AUCValues)
        
    trainPredict=(ypreds_train_wl*ypreds_cvindex_wl).sum(axis=1)
    testPredictMean=ypreds_test_wl.mean(axis=1)
    AUC1=metrics.roc_auc_score(y, trainPredict)
    
    print('Kfold number')
    print (kfoldColumnnum)
    print(AUC1)
    ypreds_train_Full_wl['CV'+ str(kfoldColumnnum)] = trainPredict
    ypreds_test_Full_wl['CV'+ str(kfoldColumnnum)] = testPredictMean

testPredictMean=np.array(ypreds_test_Full_wl)
trainPredict=np.array(ypreds_train_Full_wl)
print('First found xgBoost prediction done!')

[ 0.7208117  0.         0.       ]
[ 0.7208117   0.72395339  0.        ]
[ 0.7208117   0.72395339  0.72250358]
Kfold number
0
0.722400440202
[ 0.72839721  0.          0.        ]
[ 0.72839721  0.73022216  0.        ]
[ 0.72839721  0.73022216  0.7284556 ]
Kfold number
1
0.729009474003
[ 0.6911718  0.         0.       ]
[ 0.6911718  0.6949653  0.       ]
[ 0.6911718   0.6949653   0.69505804]
Kfold number
2
0.693720375848
[ 0.71435825  0.          0.        ]
[ 0.71435825  0.72274483  0.        ]
[ 0.71435825  0.72274483  0.72045508]
Kfold number
3
0.719132027835
[ 0.72349315  0.          0.        ]
[ 0.72349315  0.72838919  0.        ]
[ 0.72349315  0.72838919  0.72442875]
Kfold number
4
0.725413088839
[ 0.69859756  0.          0.        ]
[ 0.69859756  0.69905838  0.        ]
[ 0.69859756  0.69905838  0.69871008]
Kfold number
5
0.69874570501
[ 0.70809075  0.          0.        ]
[ 0.70809075  0.71358667  0.        ]
[ 0.70809075  0.71358667  0.71352616]
Kfold number
6
0.711687017434
[ 

'\nstr1=time.strftime("%b%d%Y%H%M%S", time.localtime())\nfilename0=\'FinalModelPrediction_xgb_logistic_step1\'+str1+\'_niter2000_AUC_0p\'+ str(int(AUC1.mean()*1e6))\nfilename=filename0+ \'.p\'\npickle.dump((trainPredict,testPredictMean,ypreds_train_wl,ypreds_test_wl,ypreds_cvindex_wl,AUCValues), \n            open(filename,\'wb\'), protocol =2) \n\nsave_train_name=filename0 +\'_wl_ypredtrain.pkl\'\nsave_test_name=filename0 +\'_wl_ypredtest.pkl\'\n'

# Logistic Regression

In [37]:
# Data assembling
#trainPredict.dump('trainPredict__xgb_logistic_step1.p')
#testPredictMean.dump('testPredictMean__xgb_logistic_step1.p')
#xtrain_quant=np.load('xtrain_quant_for_logistic_xgb.p')
#xtest_quant=np.load('xtest_quant_for_logistic_xgb.p')


xtrain=np.column_stack((xtrain_quant,trainPredict))
xtest=np.column_stack((xtest_quant,testPredictMean))


# predictions
n_iterations=500
KFoldNumber=3
kfold = StratifiedKFold(y, KFoldNumber,shuffle=True, random_state=42)
ypreds_test_wl=pd.DataFrame()
ypreds_train_wl=pd.DataFrame()
ypreds_cvindex_wl=pd.DataFrame()
AUCValues=np.zeros(KFoldNumber)

clf = linear_model.SGDClassifier(loss='log',average=True,n_iter=n_iterations)
"""
clf = linear_model.SGDClassifier(alpha=0.0031622776601683794, average=True, class_weight='auto',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0,
       learning_rate='optimal', loss='log', n_iter=500, n_jobs=1,
       penalty='elasticnet', power_t=0.5, random_state=123, shuffle=True,
       verbose=0, warm_start=False)

"""
kfoldnum=0

train_predicted=np.zeros(y.shape)

for train,cv in kfold:
    kfoldnum=kfoldnum+1
    cv_index=np.zeros(y.shape)
   
    MODEL = clf.fit(xtrain[train,:], y[train])
    preds_fulltrain = MODEL.predict_proba(xtrain)[:,1]
    preds_fulltest = MODEL.predict_proba(xtest)[:,1]
    preds_cv=preds_fulltrain[cv]
    cv_index[cv]=1
    AUC1=metrics.roc_auc_score(y[cv], preds_cv)
    ypreds_test_wl['CV'+ str(kfoldnum)] = preds_fulltest
    ypreds_train_wl['CV'+ str(kfoldnum)] = preds_fulltrain
    ypreds_cvindex_wl['CV'+ str(kfoldnum)] = cv_index
    AUCValues[kfoldnum-1]=float(AUC1)
    print(AUCValues)

trainPredict=(ypreds_train_wl*ypreds_cvindex_wl).sum(axis=1)
testPredictMean=ypreds_test_wl.mean(axis=1)
AUC1=metrics.roc_auc_score(y, trainPredict)
print(AUC1)

str1=time.strftime("%b%d%Y%H%M%S", time.localtime())
filename0='FinalModelPrediction_xgb_Logistic_final_'+str1+'_niter' + str(n_iterations)+ ' _AUC_0p'+ str(int(AUC1.mean()*1e6))
filename=filename0 +'.p'

pickle.dump((ypreds_train_wl,ypreds_test_wl,ypreds_cvindex_wl,AUCValues), 
            open(filename,'wb'), protocol =2) 

save_train_name=filename0 +'_wl_ypredtrain.pkl'
save_test_name=filename0 +'_wl_ypredtest.pkl'

pickle.dump(trainPredict,open(save_train_name,'wb'),protocol=2)
pickle.dump(testPredictMean,open(save_test_name,'wb'),protocol=2)

print('Model prediction done!')

[ 0.77288594  0.          0.        ]
[ 0.77288594  0.78028711  0.        ]
[ 0.77288594  0.78028711  0.7783516 ]
0.77712551423
Model prediction done!


In [1]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
